In [70]:
import numpy as np
from numpy import array

import base64
import io
from PIL import Image
from pickle import load

from tensorflow.keras.models import load_model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.applications.inception_v3 import preprocess_input

In [55]:
project_path = "NeuraIC/"

In [56]:
# load the tokenizer
tokenizer = load(open('tokenizer.pkl', 'rb'))
# pre-define the max sequence length (from training)
max_length = 34
# load the model
model = load_model(project_path+'model_img_cap_pad.h5')

In [57]:
def preprocess(photo):
    # Convert all the images to size 299x299 as expected by the inception v3 model
#     img = image.load_img(image_path, target_size=(299, 299))
    img = photo.resize((299, 299))
    # Convert PIL image to numpy array of 3-dimensions
    x = image.img_to_array(img)
    # Add one more dimension
    x = np.expand_dims(x, axis=0)
    # preprocess the images using preprocess_input() from inception module
    x = preprocess_input(x)
    return x

In [69]:
# Function to encode a given image into a vector of size (2048, )
def encode(image):
    image = preprocess(image) # preprocess the image
    
    InceptionV3_model = InceptionV3(weights='imagenet')
    model_iV3 = Model(InceptionV3_model.input, InceptionV3_model.layers[-2].output)
    
    fea_vec = model_iV3.predict(image) # Get the encoding vector for the image
    fea_vec = np.reshape(fea_vec, fea_vec.shape[1]) # reshape from (1, 2048) to (2048, )
    return fea_vec

In [65]:
# map an integer to a word
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

In [66]:
def greedySearch(photo):
    in_text = 'startseq'
    for i in range(max_length):
        # integer encode input sequence
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        yhat = model.predict([photo,sequence], verbose=0)
        yhat = np.argmax(yhat)
        word = word_for_id(yhat,tokenizer)
        in_text += ' ' + word
        if word == 'endseq':
            break
    final = in_text.split()
    final = final[1:-1]
    final = ' '.join(final)
    return final

In [67]:
def NeuraIC(base64_image):
    decodedImage = base64.b64decode(base64_image)
    image = Image.open(io.BytesIO(decodedImage))
    print(type(image))
    img_features = encode(image)
    print("encode complete")
    print(img_features)
    return greedySearch(img_features)

In [68]:
with open("abhi.jpg", "rb") as image_file:
    encoded_string = base64.b64encode(image_file.read())
print(NeuraIC(encoded_string))

<class 'PIL.JpegImagePlugin.JpegImageFile'>
encode complete
tf.Tensor([0.09254385 0.11425599 0.414778   ... 0.8051866  0.97208786 0.68009454], shape=(2048,), dtype=float32)


ValueError: Data cardinality is ambiguous:
  x sizes: 2048, 1
Make sure all arrays contain the same number of samples.